***
### 21-Jan-2019 stable version
The new version of the code take in considerartion cloud and water masking from all the scene and sensors (lc8 and s2a)
1. [rasterio](https://anaconda.org/conda-forge/rasterio) version = 1.0.11
***
What does the code do?
- Takes bands (nir, swir, quality bands) from lc8 and s2a compressed archives
- Masks the nir and swir bands using the quality bands
- Computes NBR band and dNBR band for each study case

### Imprting libraries and setingup working dectories

In [1]:
import rasterio, os, pprint, zipfile, re, fiona, tarfile
from rasterio.plot import show, show_hist
from rasterio import Affine
import rasterio.mask
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#wrkdir='..\\data'
wrkdir='../data2'
#rsltdir='..\\resultS2A'
rslts2a = '../rsltS2A'
rsltlc8 = '../rsltLC8'
# thezip='zip://'
# tifbands='tifbands'
posProcessdir='/media/mustapha/Seagate/NBRproject/posProcess'

### Cloud, water, ice, snow masking

In [ ]:
def clMaskValues(sat):
    if sat=='S2A':
        #scl s2a vlaues: 0:NO_DATA, 1: SATURATED-OR-DEFFECTIVE; 2:DARK-AREA-PIXELS
        #3:CLOUD_SHADOWS, 4:VEGETATION, 5:NOT_VEGETATED, 6:WATER, 7:UNCLASSIFIED
        #8:CLOUD_MEDIUM_PROBABILITY, 9:CLOUD_HIGH_PROBABILITY, 10:THIN_CIRRUS, 11:SNOW
        valuestomask= [0,1,3,6,8,9,10,11]
    elif sat=='LC8':
        fill = [1]
#         clear_land = [322, 386, 834, 898, 1346]
        clear_water = [324, 388, 836, 900, 1348]
        cloud_shadow = [328, 392, 840, 904, 1350]
        snow_ice = [336, 368, 400, 432, 848, 880, 912, 944, 1352]
        cloud = [352, 416, 480, 864, 928, 992]
        valuestomask = fill+cloud_shadow+snow_ice+cloud
    return valuestomask

In [ ]:
def cloudMaskedName(path):
    basename = os.path.basename(path)
    nsplit = basename.split('.')
    cmname ='.'.join(('_'.join((nsplit[0],'cm')),nsplit[1]))
    return os.path.join(os.path.dirname(path),cmname)
# name=  '../result2_S2A/prtoct18/20180907/S2A_prtoct18_T29SMC_20180907_B8A.tif'
# cloudMaskedName(name)

In [ ]:
def rasterCloudMask(inband, mask_band, sat):
    """ inband: band to mask
        mask_band: band cntaining the the masking value. i.e. quaity band
        outband: path to the output band
        sat: satellite used. S2A by default
    """
    #read quality band 
    with rasterio.open(mask_band,'r') as src_qb:
        qband=src_qb.read(1)
        
    #read band to mask     
    with rasterio.open(inband, 'r') as src_inb:
        src_arr = src_inb.read(1,masked=True)
        nodata = src_inb.meta['nodata']
        dtype = src_inb.meta['dtype']
        meta= src_inb.profile
            
    #list of values to mask. source: SCL band for S2A
    #calling clMaskValues function
    valuestomask = clMaskValues(sat)
    #create umpty array mask
    cl_mask = np.zeros(qband.shape)
    
    #assign value one to the mask array where Valuestomask contain the listed values
    for value in valuestomask:
        cl_mask[qband == value] = 1

#     #mask the the wanted array using the pre_mask
    src_arr_cl_free = np.ma.masked_array(src_arr, mask=cl_mask.astype(bool))
    #assign Nodata value to masked value
    src_arr_cl_free.data[src_arr_cl_free.mask == True]= nodata
    
    #calling cloudMaskedName function
    outband = cloudMaskedName(inband)

#     #PLOTING
#     fig, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3,2,figsize=(14,21))
#     show(src_arr.data, ax=ax1, title='data')
#     show(src_arr.mask, ax=ax2, title='mask')
#     show(src_arr_cl_free, ax=ax3, title='data')
#     show(src_arr_cl_free.mask, ax=ax4, title='mask')
#     show(src_arr_cl_free, ax=ax5, title='data')
#     show(src_arr_cl_free.mask, ax=ax6, title='mask')
    
    with rasterio.open(outband, 'w', **meta) as dst:
#         dst.write_band(1,arr_with_None.astype(rasterio.uint16)) #modified for landsat8
        dst.write_band(1,src_arr_cl_free.astype(dtype))
    return outband
# inband    ='../rsltS2A/prtsin/20180907/S2A_prtsin_T29SMC_20180907_B12.tif'
# mask_band ='../rsltS2A/prtsin/20180907/S2A_prtsin_T29SMC_20180907_SCL.tif'
# sat='S2A'
# inband    ='../rsltLC8/prtsin/20180922/LC8_prtsin_204033_20180922_band5.tif'
# mask_band ='../rsltLC8/prtsin/20180922/LC8_prtsin_204033_20180922_qa.tif'
# sat='LC8'
# rasterCloudMask(inband, mask_band, sat)

### NBR computing

In [ ]:
def nbrName(nir, swir):
    nirbase = os.path.basename(nir)
    swirbase = os.path.basename(swir)
    ns  = nirbase.split('.')[0].split('_')
    ss = swirbase.split('.')[0].split('_')
    nbrname = '.'.join(('_'.join((ns[0],ns[1],ns[2],ns[3],ns[4],ss[4],ns[5],'nbr')),nir.split('.')[-1]))
    return os.path.join(os.path.dirname(nir),nbrname)

# nir = '../result2_S2A/prtoct18/20180907/S2A_prtoct18_T29SMC_20180907_B8A_cm.tif'
# swir = '../result2_S2A/prtoct18/20180907/S2A_prtoct18_T29SMC_20180907_B12_cm.tif'
# nbrName(nir, swir)

In [ ]:
def NBRfuncion(NIRband,SWIRband):
    '''
    expecting three band paths, NIR, SWIR and the output (NBR)
    '''
    with rasterio.open(NIRband) as src1:
        left1, bottom1, right1, top1 = src1.bounds
#         print(left1, bottom1, right1, top1)
    with rasterio.open(SWIRband) as src2:
        left2, bottom2, right2, top2 = src2.bounds
#         print(left2, bottom2, right2, top2)
        #set common window
        left, bottom,right,top = max((left1,left2)), max((bottom1,bottom2)), min((right1,right2)), min((top1, top2))
        window1 = rasterio.windows.from_bounds(left, bottom,right,top, transform=src1.transform)
        window2 = rasterio.windows.from_bounds(left, bottom,right,top, transform=src2.transform)
        
    with rasterio.open(NIRband) as srcNIR:        
        NIRarr=srcNIR.read(1, window=window1, masked=True)
#         print('NIR profile',srcNIR.profile)
#         pprint.pprint(srcNIR.meta)
        with rasterio.open(SWIRband) as srcSWIR:
            SWIRarr = srcSWIR.read(1, window=window2, masked=True)
#             print('SWIR profile',srcSWIR.profile)
#             pprint.pprint(srcSWIR.meta)
            np.seterr(divide='ignore', invalid='ignore')            
            NBRarr = (NIRarr.astype(float) - SWIRarr.astype(float)) / (NIRarr + SWIRarr)
#             NBRarr = NBRarr.astype('float32')
#             print('NBR min',NBRarr.min(), 'NBR max',NBRarr.max())
#             print(NBRarr.data)
#             print(NBRarr.filled)
            meta = srcSWIR.profile.copy()
            meta.update({'dtype': rasterio.float32,                         
                         'width': srcSWIR.shape[1],
                         'height': srcSWIR.shape[0],
                         'transform': srcSWIR.window_transform(window2)
                        }) #this maybe changed in the future !! have to think about it
            
#             fig, ((ax1,ax2,ax3),(ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(3,3,figsize=(21,21))
#             show_hist(NIRarr, ax=ax1, title='Nir')
#             show(NIRarr.mask, ax=ax7, title='Nir Mask')
#             show_hist(SWIRarr, ax=ax2,title='swir')
#             show(SWIRarr.mask, ax=ax8,title='swir Mask')
#             show_hist(NBRarr, ax=ax3,title='NBR')
#             show(NBRarr.mask, ax=ax9,title='NBR Mask')
#             show(NIRarr, ax=ax4,title='Nir arr')
#             show(SWIRarr, ax=ax5,title='swir arr')
#             show(NBRarr, ax=ax6,title='NBR arr')
            
            nbrband = nbrName(NIRband,SWIRband)
            with rasterio.open(nbrband,'w', **meta) as dst:
                dst.write_band(1,NBRarr.astype(rasterio.float32))
#                 pprint.pprint(dst.profile)
#             print('end NBRfunction')
    return nbrband
# nir='../rsltS2A/prtsin/20180907/S2A_prtsin_T29SMC_20180907_B8A_cm.tif'
# swir='../rsltS2A/prtsin/20180907/S2A_prtsin_T29SMC_20180907_B12_cm.tif'
# nir='../rsltS2A/prtsin/20181007/S2A_prtsin_T29SMC_20181007_B8A_cm.tif'
# swir='../rsltS2A/prtsin/20181007/S2A_prtsin_T29SMC_20181007_B12_cm.tif'
# NBRfuncion(nir,swir)

### dNBR computing

In [ ]:
def dNBRName(prepath, pospath):
    prepathbase = os.path.basename(prepath)
    pospathbase = os.path.basename(pospath)
    presplit= prepathbase.split('.')[0].split('_')
    possplit= pospathbase.split('.')[0].split('_')
    
    dNBRname = '.'.join(('_'.join((presplit[0],presplit[1],presplit[2],presplit[3],possplit[3],presplit[6],'dNBR')),prepathbase.split('.')[-1]))

    newdir ='/'.join(prepath.split('/')[:-2])
    
    dNBRnewname=os.path.join(newdir,dNBRname)
    
    return dNBRnewname


In [ ]:

def dNBRfunction(preNBR, posNBR):
    '''
    preNBR: input path
    posNBR: input path
    newdir: output path
    '''
    with rasterio.open(preNBR) as src1:
        left1, bottom1, right1, top1 = src1.bounds
#         print(left1, bottom1, right1, top1)
    with rasterio.open(posNBR) as src2:

        left2, bottom2, right2, top2 = src2.bounds
#         print(left2, bottom2, right2, top2)
        left, bottom,right,top = max((left1,left2)), max((bottom1,bottom2)), min((right1,right2)), min((top1, top2))
        window1 = rasterio.windows.from_bounds(left, bottom,right,top, transform=src1.transform)
        window2 = rasterio.windows.from_bounds(left, bottom,right,top, transform=src2.transform)
    with rasterio.open(preNBR,'r') as srcPRE:        
        PREarr=srcPRE.read(1, masked=True, window=window1)
  
        
        with rasterio.open(posNBR,'r') as srcPOS:
            POSarr=srcPOS.read(1, masked=True, window=window2)
#             fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(21,10))
            meta = srcPOS.meta.copy()
            nodata = meta['nodata']
            dNBRarr = (PREarr.data - POSarr.data) # don't force to be float. it is already float 32
            #create common mask 
            cmask = np.zeros(dNBRarr.shape).astype(bool)
            cmask[POSarr.mask==True]=True
            cmask[PREarr.mask==True]=True

            #Create dNBR masked array with the common mask
            dNBRarr = np.ma.MaskedArray(dNBRarr, mask=cmask)
#             print('unique values of cmask', np.unique(cmask))
            #mask data higher than 9 and lower than -9
            dNBRarr = np.ma.MaskedArray(dNBRarr, mask=(dNBRarr> 9))
            dNBRarr = np.ma.MaskedArray(dNBRarr, mask=(dNBRarr<-9))

            #assign nodata to masked values
            dNBRarr.data[dNBRarr.mask==True]=nodata
            dNBRarr.set_fill_value(nodata)
#             print(dNBRarr_new.min(), dNBRarr_new.max())
#             print('unique values of cmask', np.unique(cmask))
#             print(dNBRarr_new.filled)
            
            #update metadata
            meta.update({'dtype': rasterio.float32,
                         'height': POSarr.shape[0],
                         'width': POSarr.shape[1],
                         'transform': srcPOS.window_transform(window2),
                         'nodata': nodata
                        })

            #create outputname
            dNBR = dNBRName(preNBR, posNBR)
#             print('created dNBR', dNBR)

#             fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(21,10))
# #             show(PREarr.mask, ax=ax1)
#             show_hist(PREarr, ax=ax1)
# #             show(POSarr.mask, ax=ax2)
#             show_hist(POSarr, ax=ax2)
# #             show(dNBRarr.mask, ax=ax3)
#             show_hist(dNBRarr, ax=ax3)
#             show(PREarr, ax=ax4)
#             show(POSarr, ax=ax5)
#             show(dNBRarr, ax=ax6)
            
        
            with rasterio.open(dNBR,'w', **meta) as dst:
                dst.write_band(1,dNBRarr.astype(rasterio.float32))
#                 pprint.pprint(dst.profile)
    return dNBR
# nbr1 = '../rsltLC8/prtps1/20171005/LC8_prtps1_204032_20171005_band5_band7_cm_nbr.tif'
# nbr2 = '../rsltLC8/prtps1/20171106/LC8_prtps1_204032_20171106_band5_band7_cm_nbr.tif'
# # nbr1 = '../rsltS2A/prtps1/20171012/S2A_prtps1_T29TNE_20171012_B8A_B12_cm_nbr.tif'
# # nbr2 = '../rsltS2A/prtps1/20171022/S2A_prtps1_T29TNE_20171022_B8A_B12_cm_nbr.tif'
# # # nbr1='../rsltLC8/prtsin/20180922/LC8_prtsin_204033_20180922_band5_band7_cm_nbr.tif'
# # # nbr2='../rsltLC8/prtsin/20181008/LC8_prtsin_204033_20181008_band5_band7_cm_nbr.tif'
# # nbr1='../rsltS2A/prtsin/20180907/S2A_prtsin_T29SMC_20180907_B8A_B12_cm_nbr.tif'
# # nbr2='../rsltS2A/prtsin/20181007/S2A_prtsin_T29SMC_20181007_B8A_B12_cm_nbr.tif'
# dNBRfunction(nbr1,nbr2)

### Reclassification to fire severity

In [ ]:
def reclassBandName(path):
    basename = os.path.basename(path)
    subname ='.'.join(('_'.join((basename.split('.')[0],'7class')),basename.split('.')[-1]))
    return os.path.join(os.path.dirname(path),subname)

In [ ]:
def dNBRreclass(dNBR):
    '''
    this function reclassifies the dNBR
    '''
    with rasterio.open(dNBR) as src:
        arr = src.read(1, masked=True)
#         print(arr.filled)
        #copy metadata
        meta = src.profile.copy()
        nodata = 0
        dtype = np.uint8
        meta.update({'dtype': dtype,
                    'nodata': nodata})
#         fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(21,10))
# #         show_hist(arr, ax=ax1)
#         show(arr.mask, ax=ax1, title='input mask')
#         show(arr, ax=ax4, title='raster input')
        
        #reclassification
        arr[(-9.00 <= arr) & (arr < -0.25)] = 11
        arr[(-0.25 <= arr) & (arr < -0.10)] = 12
        arr[(-0.10 <= arr) & (arr <  0.10)] = 13
        arr[( 0.10 <= arr) & (arr <  0.27)] = 14
        arr[( 0.27 <= arr) & (arr <  0.44)] = 15
        arr[( 0.44 <= arr) & (arr <  0.66)] = 16
        arr[( 0.66 <= arr) & (arr <= 9.00)] = 17
#         show_hist(arr, ax=ax2)
#         show(arr.mask, ax=ax2, title='mask first reclassification')
#         show(arr, ax=ax5, title='arr first reclassification')
        
        arr[arr==11]=1; arr[arr==12]=2; arr[arr==13]=3;
        arr[arr==14]=4; arr[arr==15]=5; arr[arr==16]=6; arr[arr==17]=7       

        #all data out of range will be no data
#         np.ma.masked_array(arr, mask=(arr > 7))
#         np.ma.masked_array(arr, mask=(arr < 0))
#         arr[ arr > 7 ]= nodata
#         arr[ arr < 0 ]= nodata
        #fill the masked value with Nodata value
        arr.data[arr.mask==True]=nodata
        arr.set_fill_value(nodata)
        
        arr= arr.astype(dtype)

#         print(arr.filled)
#         show_hist(arr, ax=ax3)       
#         show(arr.mask, ax=ax3, title='mask second reclass')
#         show(arr, ax=ax6, title='arr second reclass')
        #calling reclass bands
        dNBR7class = reclassBandName(dNBR)
        with rasterio.open(dNBR7class,'w', **meta) as dst:
            dst.write_band(1,arr)
#             pprint.pprint(dst.profile)
    return dNBR7class
# band = '../rsltLC8/prtoct18/LC8_prtoct18_204033_20180922_20181008_cm_dNBR.tif'
# band = '../rsltS2A/prtoct18/S2A_prtoct18_T29SMC_20180907_20181007_cm_dNBR.tif'
# band ='../rsltS2A/prtsin/S2A_prtsin_T29SMC_20180907_20181007_cm_dNBR.tif'
# dNBRreclass(band)



### sr band name listing

In [ ]:
def srBandsNameList(Sat, sourcefile):
    """ 
    Input
    Sat: S2A or LC8
    sourcefile: zipfile or tar_file
    Return list of band names
    """
    if Sat=='S2A':
        B8Agrep = re.compile('.*(B8A_20m).jp2$')
        B12grep = re.compile('.*(B12_20m).jp2$')
        SCLgrep = re.compile('.*(SCL_20m).jp2$')
        B8Aband = list(filter(lambda x: B8Agrep.search(x),sourcefile.namelist()))[0]   
        B12band = list(filter(lambda x: B12grep.search(x),sourcefile.namelist()))[0]
        SCLband = list(filter(lambda x: SCLgrep.search(x),sourcefile.namelist()))[0]
        return [B8Aband,B12band,SCLband]
    elif Sat=='LC8':
        b5grep = re.compile('.*_sr_band5.tif$')
        b7grep = re.compile('.*_sr_band7.tif$')
        qbgrep = re.compile('.*_pixel_qa.tif$')
        band5  = list(filter(lambda x: b5grep.search(x),sourcefile.getnames()))[0]
        band7  = list(filter(lambda x: b7grep.search(x),sourcefile.getnames()))[0]
        qband  = list(filter(lambda x: qbgrep.search(x), sourcefile.getnames()))[0]  
        return [band5,band7,qband]

In [ ]:
def srBandsNewNameList(Sat,firename, scene):
    if   Sat == 'S2A':
        try:
            tile, date, band= re.match(r'(T.{5})_(\d{8})T\d{6}_(B.{2}|SCL)_\d{2}m.jp2', scene).groups()
        except:
            tile, date, band= re.match(r'L2A_(T.{5})_(\d{8})T\d{6}_(B.{2}|SCL)_\d{2}m.jp2', scene).groups()
        print(tile,date,band)
    elif Sat == 'LC8':
        try:
            tile, date, band = re.match(r'LC08_L1TP_(\d{6})_(\d{8})_\d{8}_01_T1_sr_(.*)\.tif$',scene).groups()
        except:
            tile, date, band = re.match(r'LC08_L1TP_(\d{6})_(\d{8})_\d{8}_01_T1_pixel_(qa)\.tif$',scene).groups()
    band_new_name = '.'.join(('_'.join((Sat,firename,tile,date,band)),'tif'))
    return (band_new_name, date)


### Clip functions

In [ ]:
def clipBandName2(band,aoi):
    """
    New naming for multiple fire per image 
    band:
    aoi: i.e. firenamedate_aoi01.shp. the "aoi01" will be used in the fire name
    """
    baseband = os.path.basename(band)
    baseaoi = os.path.basename(aoi)
    subaoi=baseaoi.split('.')[0].split('_')[-1]
#     print(subaoi)
    subname ='.'.join(('_'.join((baseband.split('.')[0],subaoi)),baseband.split('.')[-1]))
    #postprocessing directory
    pp_dir = ''.join((baseband.split('.')[0].split('_')[1],subaoi))
        
    return (pp_dir,subname)

In [ ]:
def rasterClip(inband,aoi,dstsub):
    '''
    inband: band to clip
    aoi: AOI
    dst: destination directory
    '''        
    with fiona.open(aoi) as srcV:
    #     pprint.pprint(src3[0]["geometry"])
        features = [feature["geometry"] for feature in srcV]
        srcV_crs = srcV.crs
#         print('vector',srcV_crs)
        with rasterio.open(inband) as srcR:
            srcR_crs = srcR.crs.to_dict()

            assert srcV_crs == srcR_crs
            
            nodata = srcR.meta['nodata']
    #         print('raster:', srcR_crs)
            out_image, out_transform = rasterio.mask.mask(srcR, features, crop=True, nodata=nodata)
            out_meta = srcR.meta.copy() 

            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})

            with rasterio.open(dstsub, "w", **out_meta) as dst:
                dst.write(out_image)
#                 plt.imshow(out_image[0])

    return dstsub
# inband ='../posProcess/prtsinaoi01/LC8_prtsin_204033_20180922_band5_band7_cm_nbr_aoi01_qgis.tif'
# inband ='../posProcess/prtsinaoi01/LC8_prtsin_204033_20180922_band5_band7_cm_nbr_aoi01_qgis.tif'

In [ ]:
def clipMultiple(band,srcdir,dstdir):
    """ Use rasterClip function to clip a list of aoi from a unique raster
    band: band to clip
    aoilist: list of aoi
    dstdir: postprocessing dir
    """
    aoigrep = re.compile('.*\.shp$')
    aoilist = list(filter(lambda x: aoigrep.search(x),os.listdir(os.path.join(srcdir))))
    assert len(aoilist) >= 1, 'No aoi in the directory'
    for aoi in aoilist:
        
        #create sub dst path
        pp_dir, subname = clipBandName2(band,aoi)
        
        if not os.path.exists(os.path.join(dstdir,pp_dir)):
            os.makedirs(os.path.join(dstdir,pp_dir))
        dstsub = os.path.join(dstdir,pp_dir,subname)
        #clip
        aoipath=os.path.join(srcdir,aoi)
        rasterClip(band,aoipath,dstsub)
        

### S2A functions

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling 

def s2aResample(inputpath, outputpath,res='30'):
    '''
    inputpath : bands path. in this case it's the path of the band inside the Zipfile
                'zip://data/files.zip!RGB.byte.jp2'
    outputpath: 
    res: str indicating the output resolution 
          i.e. res='30'  for 30m resolution   
    '''
    with rasterio.open(inputpath,'r') as src:
        src_arr = src.read(1, masked=True)
#         print('RESAMPLING input jp2 profile \n', src.profile)
        new_resolution = (int(res),int(res))
        notdata = -9999.0
        new_transform, new_width, new_height = calculate_default_transform(src.crs, src.crs,
                                                                           src.width, src.height,
                                                                           *src.bounds,
                                                                           resolution=new_resolution)

        #creating a empty contener for the 30m resolution S2A raster    
        dst_arr = np.zeros(shape=(new_width,new_height), dtype=np.int16)#change nodata to -9999
        #update metadata
        meta = src.profile.copy()
        meta.update({'transform': new_transform,
                     'width' : new_width,
                     'height': new_height,
                     'nodata': notdata,#change nodata to -9999
                     'driver': 'GTiff',
                     'dtype' : 'int16'})#change nodata to -9999
#                      'tiled': True,
#                      'blockxsize': 256,
#                      'blockysize': 256,
#                      'photometric': 'YCbCr',
#                      'compress':'JPEG' })   #'affine':new_transform, was removed
        
#         print('RESAMPLING output tif profile \n', meta)
        reproject(src_arr, dst_arr,
                  src_transform=src.transform,
                  dst_transform=new_transform,
                  width=new_width,
                  height=new_height,
                  src_crs=src.crs,# no change in the CRS
                  dst_crs=src.crs,
                  resample=Resampling.average) 
        
        #creating mask where data == 0
        dst_arr[dst_arr==0]=notdata#change nodata to -9999
        dst_arr = np.ma.masked_where(dst_arr==notdata,dst_arr)#change nodata to -9999
        dst_arr.set_fill_value(notdata)#change nodata to -9999
        
        with rasterio.open(outputpath,'w', **meta) as dst:
            dst.write_band(1,dst_arr.astype(rasterio.int16))#change nodata to -9999

In [ ]:
def unarchiveS2ABands(s2azipdir, Sat='S2A'):
    """
    return: list of the resampled and saved S2A bands
    """
    
    with zipfile.ZipFile(s2azipdir, 'r') as zip_file:

        #calling srBandsNameList function
        s2a_bands_list = srBandsNameList(Sat, zip_file)
        
        #umpty band list for stacking
        list_extracted_bands=[]

        for bandpath in s2a_bands_list:
            #Input path for rasterio (Apache path style 'zip://dir/file.zip!bandpath)
            thezip='zip://'
            jp2bandpath=thezip+s2azipdir+'!/'+bandpath
        
           #calling srBandsNewNameList function
            band_new_name,datetxt =srBandsNewNameList(Sat,cntr, os.path.basename(jp2bandpath))
            
            if not os.path.exists(os.path.join(rslts2a,cntr,datetxt)):
                os.makedirs(os.path.join(rslts2a,cntr,datetxt))
            
            tifbandpath = os.path.join(rslts2a,cntr,datetxt,band_new_name)
            print(tifbandpath)

            #calling the s2aResample function
            s2aResample(jp2bandpath, tifbandpath) #calling the s2aResample function
            #create list of extracted band paths in the destination directory
            list_extracted_bands.append(tifbandpath)   
            
#             #cliping the resampled raster band
#             rasterClip(tifbandpath,aoiPath)
            srcdir = os.path.join(wrkdir,cntr)
            clipMultiple(tifbandpath,srcdir,posProcessdir)
        
    return list_extracted_bands

### Main S2A _ generates the output

In [ ]:
cntrlist = [ name for name in os.listdir(wrkdir) if os.path.isdir(os.path.join(wrkdir, name)) ]
print('list of fire cases',cntrlist)
cntrlist=[ 'itatug']
for cntr in cntrlist:
    print('PROCESSSING', cntr)

    # listing the zip file path
    s2agrep=re.compile('S2A.*\.zip$')
    s2aziplist= list(filter(lambda x: s2agrep.search(x),os.listdir(os.path.join(wrkdir,cntr))))
    print(s2aziplist)
    if len(s2aziplist) == 0:
        continue
    assert len(s2aziplist)== 2, 'list of s2a zip archive must be equal to 2'
    #umpty list of NBR bands
    
    list_nbr_bands=[]
    
    for s2azip in s2aziplist:
        
        s2azipdir = os.path.join(wrkdir,cntr,s2azip)
        
        #calling unarchiveS2ABands function
        list_extracted_bands = unarchiveS2ABands(s2azipdir)
        print('list_extracted_bands', list_extracted_bands)
         
        #cloud masking
        list_clmasked_bands=[]
        for band in list_extracted_bands[:2]:
            
            #calling the cloud masking function
            cmband = rasterCloudMask(band,list_extracted_bands[-1],'S2A')
            #update cloud masked band list
            list_clmasked_bands.append(cmband)
            
            #clip cloud masked bands
#             rasterClip(cmband, aoiPath)
            srcdir = os.path.join(wrkdir,cntr)
            clipMultiple(cmband,srcdir,posProcessdir)
        #calling the NBRfunction        
        nbrband = NBRfuncion(list_clmasked_bands[0],list_clmasked_bands[1])
        #update nbr band list
        list_nbr_bands.append(nbrband)
        
        #clip nbr bands
#         rasterClip(nbrband, aoiPath)
        srcdir = os.path.join(wrkdir,cntr)
        clipMultiple(nbrband,srcdir,posProcessdir)
#         
    #calling dNBR function
    list_nbr_bands.sort
    pprint.pprint(list_nbr_bands)
    
    dNBRband = dNBRfunction(list_nbr_bands[0], list_nbr_bands[1])   
    #clip dNBR band
    srcdir = os.path.join(wrkdir,cntr)
    clipMultiple(dNBRband,srcdir,posProcessdir)
    
    #calling the dNBR reclassification function
    print('dNBRband', dNBRband)
    dNBRrecl = dNBRreclass(dNBRband)

    #calling rasterClip function
    srcdir = os.path.join(wrkdir,cntr)
    clipMultiple(dNBRrecl,srcdir,posProcessdir)

print('Process ended successfully')

### LC8 functions

In [ ]:
from rasterio.warp import calculate_default_transform,reproject,Resampling

def lc8UpdateReproject (band,lc8update):
    #the aoi is used as CRS reference, not for croping
    #lc8 band CRS is compared to aoi's CRS
    aoigrep = re.compile('.*\.shp$')
    aoilist = list(filter(lambda x: aoigrep.search(x),os.listdir(os.path.dirname(lc8targzarchive))))
    assert len(aoilist) >= 1, 'No aoi in the directory'
    aoipath=os.path.join(os.path.dirname(lc8targzarchive),aoilist[0])
    with fiona.open(aoipath) as srcV:
        srcV_crs = srcV.crs
        
        with rasterio.open(band) as src:
            lc8arr = src.read(1, masked=True)
            #UPDATE LC8 RASTER VALUES to fit S2A RASTER VALUES
            meta=src.profile
            nodata=meta['nodata']
            dtype=meta['dtype']
            
            #mask value Nodata value
            lc8arr = np.ma.masked_where(lc8arr<=0,lc8arr)#change nodata to -9999
            dst_arr=lc8arr.copy()

            #REPROJECT in case lc8 CRS is different than the aoi CRS ( the reference)
            if srcV_crs != src.crs:
                transform, width, height = calculate_default_transform(src.crs,srcV_crs,
                                                                       src.width,src.height,
                                                                       *src.bounds)
                meta.update({'crs': srcV_crs,
                             'transform': transform,
                             'width': width,
                             'height': height})
#                 dst_arr = np.zeros(shape=(height,width), dtype=np.uint16)
                dst_arr = np.zeros(shape=(height,width), dtype=dtype)#change nodata to -9999
                reproject(lc8arr, dst_arr,
                      src_transform=src.transform,
                      dst_transform=transform,
                      width=width,
                      height=height,
                      src_crs=src.crs,
                      dst_crs=srcV_crs,
                      resample=Resampling.nearest) 
        
        with rasterio.open(lc8update,'w', **meta) as dst:
            dst.write_band(1,dst_arr.astype(dtype))#change nodata to -9999
    return lc8update

In [ ]:
def unarchiveLC8Bands(lc8targzarchive,dst_dir):
    """ extracting the LC8 bands
    lc8targzarchive: lc8  tar archive
    extractdir: directory to extract to
    return:     sr_nbr.tif and _pixel_qa.tif
    """
    # open tar file
    tar_file = tarfile.open(lc8targzarchive,'r') #open tarfile
    
    #calling srBandsNameList function
    lc8_bands_list = srBandsNameList('LC8', tar_file)
    print(lc8_bands_list)
#     nbrpath = os.path.join(extrcatdir,nbrband)
#     qbpath  = os.path.join(extrcatdir,qband)
    for band in lc8_bands_list:
    
        #start the extraction
        tar_file.extract(band, path=dst_dir)
    tar_file.close()# close tarfile
    list_new_names=[]
    for band in lc8_bands_list:
        #rename landsat bands 
        band_new_name,datetxt = srBandsNewNameList('LC8',cntr,band)
        
        #create date directory
        if not os.path.exists(os.path.join(rsltlc8,cntr,datetxt)):
                os.makedirs(os.path.join(rsltlc8,cntr,datetxt))
        #rename and relocate the file with lc8 new parameters
        lc8updated = os.path.join(dst_dir,datetxt,band_new_name)
        #calling the lc8Update function
        extractedlc8band = os.path.join(rsltlc8,cntr,band)
#         lc8Update (extractedlc8band,lc8updated)
        #look for aoi to take CRS from it

        lc8UpdateReproject (extractedlc8band,lc8updated)
        os.remove(os.path.join(dst_dir,band))
        list_new_names.append(lc8updated)

        #clip lc8 band
#         rasterClip(lc8updated,aoiPath)
        srcdir = os.path.join(wrkdir,cntr)
        clipMultiple(lc8updated,srcdir,posProcessdir)
        
    return list_new_names
    

### Main LC8 _ generates the output

In [ ]:
cntrlist = [ name for name in os.listdir(wrkdir) if os.path.isdir(os.path.join(wrkdir, name)) ]
print(cntrlist)
# cntrlist=[ 'itatug', 'prtped', 'prtps1', 'prtps2', 'prtps3', 'prtsin']
cntrlist=[ 'itatug']
for cntr in cntrlist:
    print('PROCESSSING', cntr)
    
    if not os.path.exists(os.path.join(rsltlc8,cntr)):
        os.makedirs(os.path.join(rsltlc8,cntr))  
    # defining the tar file path
    lc8targzgrep = re.compile('LC08.*\.tar.gz$')
    lc8targzlist = list(filter(lambda x: lc8targzgrep.search(x),os.listdir(os.path.join(wrkdir,cntr))))
    #define the output directory
#     extractdir=os.path.join(rsltlc8,cntr)
    if len(lc8targzlist) == 0:
        continue
        
    #umpty list of NBR bands
    list_nbr_bands=[]
    
    for lc8targz in lc8targzlist:
        lc8targzarchive = os.path.join(wrkdir,cntr,lc8targz)
        dst_dir = os.path.join(rsltlc8,cntr)
        # open tar file
        list_extracted_bands = unarchiveLC8Bands(lc8targzarchive,dst_dir)
                #cloud masking
        list_clmasked_bands=[]

        for band in list_extracted_bands[:2]:
            
            #calling the cloud masking function
            cmband = rasterCloudMask(band,list_extracted_bands[-1],'LC8')
            #update cloud masked band list
            list_clmasked_bands.append(cmband)
            
            #clip cloud masked bands
#             rasterClip(cmband, aoiPath)
            srcdir = os.path.join(wrkdir,cntr)
            clipMultiple(cmband,srcdir,posProcessdir)
        #calling the NBRfunction        
        nbrband = NBRfuncion(list_clmasked_bands[0],list_clmasked_bands[1])
        #update nbr band list
        list_nbr_bands.append(nbrband)
        
        #clip nbr bands
#         rasterClip(nbrband, aoiPath)
        srcdir = os.path.join(wrkdir,cntr)
        clipMultiple(nbrband,srcdir,posProcessdir)
#         
    #calling dNBR function
    list_nbr_bands.sort
    pprint.pprint(list_nbr_bands)
    
    dNBRband = dNBRfunction(list_nbr_bands[0], list_nbr_bands[1])   
    #clip dNBR band
    srcdir = os.path.join(wrkdir,cntr)
    clipMultiple(dNBRband,srcdir,posProcessdir)
    
    #calling the dNBR reclassification function
    print('dNBRband', dNBRband)
    dNBRrecl = dNBRreclass(dNBRband)

    #calling rasterClip function
    srcdir = os.path.join(wrkdir,cntr)
    clipMultiple(dNBRrecl,srcdir,posProcessdir)

print('Process ended successfully')